# BB84 Quantum Key Distribution Simulation

This simulates the BB84 protocol for secure key exchange between Alice and Bob, with optional Eve eavesdropping. It uses quantum states for encoding bits.

## Protocol Overview:
1. Alice sends qubits in random bases (rectilinear or diagonal).
2. Bob measures in random bases.
3. They compare bases publicly and keep matching measurements as key.
4. Check for eavesdropping via error rate.

Tech: Qiskit, Aer Simulator.

In [7]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

# Use matrix_product_state method — perfect for large qubit counts with low/no entanglement (like BB84)
simulator = AerSimulator(method='matrix_product_state')

def encode_bits(bits, bases):
    n = len(bits)
    qc = QuantumCircuit(n)
    for i in range(n):
        if bits[i] == 1:
            qc.x(i)
        if bases[i] == 1:
            qc.h(i)
    return qc

def measure_bits(qc, bases):
    for i in range(len(bases)):
        if bases[i] == 1:
            qc.h(i)
    
    qc.measure_all(add_bits=True)
    
    # Skip transpile — run raw circuit directly (avoids coupling map checks)
    result = simulator.run(qc, shots=1).result()
    counts = result.get_counts()
    
    bitstring = list(counts.keys())[0]  # e.g., '0101...'
    # Reverse because Qiskit bitstrings are little-endian (qubit 0 = rightmost)
    measured_bits = [int(bit) for bit in bitstring[::-1]]
    
    return measured_bits

# Rest of bb84_protocol unchanged...
def bb84_protocol(key_length, eavesdrop=False):
    alice_bits = np.random.randint(2, size=key_length)
    alice_bases = np.random.randint(2, size=key_length)
    
    qc = encode_bits(alice_bits, alice_bases)
    
    if eavesdrop:
        eve_bases = np.random.randint(2, size=key_length)
        eve_bits = measure_bits(qc.copy(), eve_bases)
        qc = encode_bits(eve_bits, eve_bases)
    
    bob_bases = np.random.randint(2, size=key_length)
    bob_bits = measure_bits(qc.copy(), bob_bases)
    
    matching = [i for i in range(key_length) if alice_bases[i] == bob_bases[i]]
    alice_key = [int(alice_bits[i]) for i in matching]
    bob_key = [int(bob_bits[i]) for i in matching]
    
    if len(alice_key) == 0:
        return [], [], 0.0
    
    sample_size = max(1, len(alice_key) // 4)
    sample_idx = np.random.choice(len(alice_key), sample_size, replace=False)
    errors = sum(alice_key[i] != bob_key[i] for i in sample_idx)
    error_rate = errors / sample_size
    
    return alice_key, bob_key, error_rate

# Tests
alice_key, bob_key, error_rate = bb84_protocol(100)
print(f"Alice's key (first 10): {alice_key[:10]}")
print(f"Bob's key (first 10): {bob_key[:10]}")
print(f"Error rate (no Eve): {error_rate:.4f}")

alice_key_eve, bob_key_eve, error_rate_eve = bb84_protocol(100, eavesdrop=True)
print(f"\nWith Eve - Error rate: {error_rate_eve:.4f}")

Alice's key (first 10): [1, 0, 1, 0, 1, 0, 0, 1, 1, 0]
Bob's key (first 10): [1, 0, 1, 0, 1, 0, 0, 1, 1, 0]
Error rate (no Eve): 0.0000

With Eve - Error rate: 0.2727


## Results
Without Eve, error rate ~0. With Eve, it's higher (~0.25), detecting intrusion.